# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results and you'll learn how to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Use stepwise selection methods to determine the most important features for a model
* Use recursive feature elimination to determine the most important features for a model

## The Ames Housing Data once more

In [7]:
%conda upgrade statsmodels

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::gensim==3.7.3=py36h0a44026_0
  - defaults/osx-64::h5py==2.9.0=py36h3134771_0
  - conda-forge/noarch::jupyter_contrib_core==0.3.3=py_2
  - conda-forge/osx-64::jupyter_contrib_nbextensions==0.5.1=py36_0
  - conda-forge/osx-64::jupyter_highlight_selected_word==0.2.0=py36_1000
  - conda-forge/osx-64::jupyter_latex_envs==1.4.4=py36_1000
  - conda-forge/osx-64::jupyter_nbextensions_configurator==0.4.1=py36_0
  - defaults/osx-64::mkl_fft==1.0.14=py36h5e564d8_0
  - defaults/osx-64::mkl_random==1.1.0=py36ha771720_0
  - defaults/osx-64::nb_conda==2.2.1=py36_0
  - defaults/osx-64::nb_conda_kernels==2.2.1=py36_0
  - defaults/noarch::nbconvert==5.5.0=py_0
  - defaults/osx-64::notebook==5.7.8=py36_0
  - defaults/osx-64::numpy==1.16.5=py36hacdab7b_0
  - defaults/osx-64::patsy==0.5.1=py36_0
  - defaults/osx-64::py-xgboost==0.90=py3

In [8]:
import pandas as pd
import numpy as np

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

## Perform stepwise selection

The function for stepwise selection is copied below. Use this provided function on your preprocessed Ames Housing data.

In [9]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [12]:
# Your code here

# X = preprocessed.drop('SalePrice_log', axis=1)
# y = preprocessed['SalePrice_log']

# result = stepwise_selection(X, y, verbose = True)
# result

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

result = stepwise_selection(X, y, verbose = True)
print('resulting features:')
print(result)

/Users/spags/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


Add  GrLivArea_log                  with p-value 1.59847e-243
Add  KitchenQual_TA                 with p-value 1.56401e-67
Add  1stFlrSF_log                   with p-value 7.00069e-48
Add  KitchenQual_Fa                 with p-value 1.70471e-37
Add  Neighborhood_OldTown           with p-value 3.20105e-23
Add  KitchenQual_Gd                 with p-value 4.12635e-21
Add  Neighborhood_Edwards           with p-value 9.05184e-17
Add  Neighborhood_IDOTRR            with p-value 1.10068e-18
Add  LotArea_log                    with p-value 1.71728e-13
Add  Neighborhood_NridgHt           with p-value 7.05633e-12
Add  BldgType_Duplex                with p-value 4.30647e-11
Add  Neighborhood_NAmes             with p-value 2.25803e-09
Add  Neighborhood_SWISU             with p-value 5.40743e-09
Add  Neighborhood_BrkSide           with p-value 8.79638e-10
Add  Neighborhood_Sawyer            with p-value 6.92011e-09
Add  Neighborhood_NoRidge           with p-value 5.87105e-08
Add  Neighborhood_Somer

ValueError: list.remove(x): x not in list

### Build the final model again in Statsmodels

In [11]:
# Your code here

import statsmodels.api as sm

x_final = X[result]
x_with_intercept = sm.add_constant(x_final)
model = sm.ols(y, x_with_intercept).fit()
model.summary()

NameError: name 'result' is not defined

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [16]:
# Your code here
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select = 5)
selector = selector.fit(X, y.values.ravel())
selector.support_

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False])

Fit the linear regression model again using the 5 selected columns

In [19]:
# Your code here
selected_columns = X.columns[selector.support_]
linreg.fit(X[selected_columns], y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Now, predict $\hat y$ using your model. You can use `.predict()` in scikit-learn. 

In [22]:
# Your code here
y_predictions = linreg.predict(X[selected_columns])
print(y_predictions)
len(y_predictions)

[0.10023007 0.10023007 0.10023007 ... 0.10023007 0.10023007 0.10023007]


1460

Now, using the formulas of R-squared and adjusted R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [31]:
# Your code here = Messed up somewhere

ss_residual = np.sum((y_predictions) **2)
ss_total = np.sum(((y - np.mean(y))**2))
r_squared = 1 -(ss_residual/ss_total)
r_squared_adj = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X[selected_columns].shape[1]-1)

print("r_squared is {}".format(r_squared))
print("r_squared_adj is {}".format(r_squared_adj))

# SS_Residual = np.sum((y_predictions)**2)
# SS_Total = np.sum((y-np.mean(y))**2)
# r_squared = 1 - (float(SS_Residual))/SS_Total
# adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X[selected_columns].shape[1]-1)

# r_squared


# r_squared is 0.239434  
# adjusted_r_squared is 0.236818

r_squared is 0.7605658182288547
r_squared_adj is 0.7597424544676059


## Level up (Optional)

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted R-squared!
- Tweak the code in the `stepwise_selection()` function written above to just perform forward selection based on the p-value 

## Summary
Great! You practiced your feature selection skills by applying stepwise selection and recursive feature elimination to the Ames Housing dataset! 